In [15]:
import os
os.chdir('/home/golopes/mestrado/projetos/FoodAid/')
import pandas as pd
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords') # install NLTK data to home user directory
import unidecode
from gensim.models import Word2Vec
from ingredient_parser import ingredient_parser
import embeddings as emb
from sklearn.metrics.pairwise import cosine_similarity



In [16]:
#####
# TREINAR MODELO
#####

csv_file = "csv_file/recipes.csv"

# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

data = pd.read_csv(csv_file)
columns=data["title"] +','+ data["ingredients"]

data["parsed"] = columns.apply(
    lambda x: ingredient_parser(x)
)

# print(data["parsed"])
data.head()

corpus = get_and_sort_corpus(data)
print(f"Length of corpus: {len(corpus)}")
        
model = Word2Vec(corpus, sg=0, workers=8, window=4, min_count=1, vector_size=100,compute_loss=True)
# Loss
print(model.get_latest_training_loss())
model.save("NLP/model/model_title_recipe.model")
#Summarize vocabulary
words = list(model.wv.index_to_key)
words.sort()


Length of corpus: 99
9730.7177734375


In [17]:
def get_and_sort_corpus(data):
    """
    Get corpus with the documents sorted in alphabetical order
    """
    corpus_sorted = []
    for doc in data.parsed:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ingredient.split()

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

def get_recommendations(N, scores,csv_file,lerning_param):
    """
    Top-N recomendations order by score
    """
    # load in recipe dataset
    df_recipes = pd.read_csv(csv_file)
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    if "title" not in lerning_param:
        lerning_param.insert(0, "title")
    lerning_param.append("score")
    recommendation = pd.DataFrame(columns=lerning_param)

    for i in top:
        # recommendation.at[i, "title"] = unidecode.unidecode(df_recipes["title"][i])
        for param in lerning_param[0:-1]:
            recommendation.at[i, param] = unidecode.unidecode(
                df_recipes[param][i]
            )
        recommendation.at[i, "score"] = f"{scores[i]}"
    return recommendation

In [18]:
#####
# TESTAR COM INPUT
#####
n_sugestions = 5
key_words = "francesinha"

# vec_tr = emb.MeanEmbeddingVectorizer(model)
vec_tr = emb.TfidfEmbeddingVectorizer(model)
    
vec_tr.fit(corpus)
doc_vec = vec_tr.transform(corpus)
doc_vec = [doc.reshape(1, -1) for doc in doc_vec]

# create tokens with elements
input = key_words.split(",")
# parse ingredient list
input = ingredient_parser(input)
# get embeddings for ingredient doc
input_embedding = vec_tr.transform([input])[0].reshape(1, -1)

# get cosine similarity between input embedding and all the document embeddings
cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
scores = list(cos_sim)
# Filter top N recommendations
learning_param=["title","ingredients","recipe"]
recommendations = get_recommendations(n_sugestions, scores,csv_file,learning_param)
recommendations


,title,ingredients,recipe,score
19,Francesinha,"bread, steak, linguica , sausage, ham, cheese,...","In a pan, saute the garlic with olive oil and ...",0.31636571884155273
69,Prego Burger,"burger buns,beef,smoked paprika,garlic powder,...","In a mixing bowl, combine the ground beef with...",0.22019627690315247
96,Rock Soup,"1 lb. beans, soaked overnight, 1 lb. Portugues...","In a large pot, add the soaked beans, chourico...",0.17354120314121246
25,Pastel de Nata com Doce de Leite,"puff pastry, milk, egg yolks, sugar,flour, cin...","Preheat the oven to 220degC. In a pot, mix the...",0.13772372901439667
95,Rock Soup Alentejana,"1 lb. beans, soaked overnight, 1 lb. potatoes,...","In a large pot, add the soaked beans, beef, ch...",0.12293700873851776
